In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Otimização do Processamento de Planilhas de Ações**

Este notebook realiza a limpeza e fusão de dados de ações de duas planilhas, aplica correções específicas e salva o resultado final em uma só planilha.

In [ ]:
# Importação da biblioteca
import pandas as pd
import numpy as np

## **1. Carregamento dos Dados Iniciais**

Carregamos as duas planilhas limpas (`stock_part1_cleaned.csv` e `stock_part2_cleaned.csv`) que contêm as partes dos dados a serem mescladas.

In [ ]:
# Carrega a primeira planilha limpa
df1 = pd.read_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part1_cleaned.csv')

# Carrega a segunda planilha limpa
df2 = pd.read_csv('/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_part2_cleaned.csv')

## **2. Verificação Inicial dos DataFrames**

Realiza uma verificação de segurança das colunsa e Dataframes carregados.

In [ ]:
# Exibe informações das colunas para confirmação
print("--- Colunas da planilha stock_part1_cleaned.csv ---")
print(f"Colunas: {df1.columns.tolist()}")
if 'ID' in df1.columns:
    print(f"Tipo de dado df1['ID']: {df1['ID'].dtype}")
else:
    print("Coluna 'ID' não encontrada em df1.") # Verifica se a coluna ID que vai ser usada para juntar as duas planilhas não está na planilha tratada 1 (Possível erro)

print("\n--- df2 (stock_part2_cleaned.csv) ---")
print(f"Colunas: {df2.columns.tolist()}")
if 'ID' in df2.columns:
    print(f"Tipo de dado df2['ID']: {df2['ID'].dtype}")
else:
    print("Coluna 'ID' não encontrada em df2.") # Verifica se a coluna ID que vai ser usada para juntar as duas planilhas não está na planilha tratada 2 (Possível erro)

--- Colunas da planilha stock_part1_cleaned.csv ---
Colunas: ['ID', 'Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividend']
Tipo de dado df1['ID']: int64

--- df2 (stock_part2_cleaned.csv) ---
Colunas: ['ID', 'Sector', 'Industry', 'Exchange', 'Market Cap (USD)', 'PE Ratio', 'EPS (USD)', 'Next Earnings Date', 'Previous Close (USD)', 'Day Low (USD)', 'Day High (USD)']
Tipo de dado df2['ID']: int64


## **3. Merge dos DataFrames**

Realizamos o merge dos dois DataFrames (`df1` e `df2`) utilizando a coluna 'ID' como chave. Usamos um 'left merge' para manter todas as linhas de `df1` e adicionar as informações correspondentes de `df2`.

In [ ]:
# Realiza o merge usando a coluna 'ID' como chave
# 'how=left' mantém todas as linhas de df1 e adiciona as colunas de df2 onde houver correspondência de 'ID'
if 'ID' in df1.columns and 'ID' in df2.columns:
    merged_df = pd.merge(df1, df2, on='ID', how='left')
    print("\nMerge realizado com sucesso.")

else:
    # Verificação de possível erro
    print("\nColuna 'ID' não encontrada em um ou ambos os DataFrames. Merge não pode ser realizado.")
    merged_df = pd.DataFrame()


Merge realizado com sucesso.


## **4. Definição das Correções**

Criamos um DataFrame contendo as informações corretas para 'Company', 'Sector' e 'Industry' para um conjunto específico de 'Tickers'. Esses dados serão usados para atualizar o DataFrame mesclado.

In [ ]:
# DataFrame com as correções de Company, Sector e Industry
correcoes = pd.DataFrame({
    "Ticker": [
        "ZM", "PYPL", "BABA", "SHOP", "META", "INTC", "WORK", "CSCO", "SNAP", "IBM",
        "AAPL", "SQ", "AMD", "TSLA", "ORCL", "SPOT", "NVDA", "LYFT", "MSFT", "ADBE",
        "TWTR", "PINS", "VMW", "GOOG", "NFLX", "DOCU", "AMZN", "CRM", "INTU", "UBER"
    ],
    "Company": [
        "Zoom Video Communications", "PayPal Holdings", "Alibaba Group", "Shopify", "Meta Platforms",
        "Intel Corporation", "Slack Technologies", "Cisco Systems", "Snap Inc.", "International Business Machines",
        "Apple Inc.", "Block, Inc.", "Advanced Micro Devices", "Tesla, Inc.", "Oracle Corporation",
        "Spotify Technology", "NVIDIA Corporation", "Lyft, Inc.", "Microsoft Corporation", "Adobe Inc.",
        "Twitter, Inc.", "Pinterest, Inc.", "VMware, Inc.", "Alphabet Inc. (Google)", "Netflix, Inc.",
        "DocuSign, Inc.", "Amazon.com, Inc.", "Salesforce, Inc.", "Intuit Inc.", "Uber Technologies"
    ],
    "Sector": [
        "Technology", "Technology", "Consumer Discretionary", "Technology", "Technology",
        "Technology", "Technology", "Technology", "Communication Services", "Technology",
        "Technology", "Technology", "Technology", "Consumer Discretionary", "Technology",
        "Communication Services", "Technology", "Consumer Discretionary", "Technology", "Technology",
        "Communication Services", "Communication Services", "Technology", "Technology", "Communication Services",
        "Technology", "Consumer Discretionary", "Technology", "Technology", "Consumer Discretionary"
    ],
    "Industry": [
        "Software", "Software", "Internet & Direct Marketing Retail", "Software", "Software",
        "Semiconductors", "Software", "Communications Equipment", "Interactive Media & Services", "IT Consulting & Other Services",
        "Consumer Electronics", "Software", "Semiconductors", "Automobiles", "Software",
        "Interactive Media & Services", "Semiconductors", "Ground Transportation", "Software", "Software",
        "Interactive Media & Services", "Interactive Media & Services", "Systems Software", "Interactive Media & Services", "Entertainment",
        "Software", "Internet & Direct Marketing Retail", "Software", "Software", "Ground Transportation"
    ]
})

## **5. Aplicação das Correções**

Agora, aplicamos as correções ao DataFrame `merged_df`.
1. Criamos uma cópia de `merged_df` para não alterar o original diretamente.
2. Padronizamos a coluna 'Ticker'
3. Iteramos sobre o DataFrame `correcoes` e atualizamos as colunas 'Company', 'Sector' e 'Industry' no DataFrame copiado para os tickers correspondentes.

In [ ]:
# Criar cópia de merged_df para aplicar correções, mantendo o original intacto se necessário.
df_corrigido = merged_df.copy()

# Verificar se a coluna 'Ticker' existe e realizar a limpeza dos tickers
if 'Ticker' in df_corrigido.columns:
    df_corrigido["Ticker"] = df_corrigido["Ticker"].str.strip().str.upper()
    for col_para_atualizar in ["Company", "Sector", "Industry"]:
        if col_para_atualizar not in df_corrigido.columns:
            df_corrigido[col_para_atualizar] = pd.NA # ou np.nan

    # Aplicar correções de Company, Sector e Industry
    for _, row in correcoes.iterrows():
        ticker_a_corrigir = row["Ticker"]
        mascara_ticker = (df_corrigido["Ticker"] == ticker_a_corrigir)

        # Atualiza as colunas 'Company', 'Sector', e 'Industry' para os tickers correspondentes
        df_corrigido.loc[mascara_ticker, ["Company", "Sector", "Industry"]] = row[["Company", "Sector", "Industry"]].values
else:
    print("Coluna 'Ticker' não foi encontrada no DataFrame mesclado.")

## **6. Salvamento do DataFrame Final Corrigido**

O DataFrame `df_corrigido`, agora contendo os dados mesclados e atualizados, é salvo em um novo arquivo CSV.

In [ ]:
caminho_arquivo_final = '/content/drive/MyDrive/Projeto Dados CITI (Squad Dani)/Databases/stock_data_merged.csv'
df_corrigido.to_csv(caminho_arquivo_final, index=False)

# Classificação das indústrias e setores pelos tickers
1. Identificação das Empresas pelos Tickers.
Cada empresa da sua lista foi identificada inicialmente por seu ticker (símbolo de negociação na bolsa), como AAPL para Apple ou ZM para Zoom. Isso é fundamental para garantir que estamos falando da empresa correta, já que nomes podem ser semelhantes entre companhias diferentes.

2. Consulta a Fontes Confiáveis
Utilizamos fontes amplamente aceitas no mercado financeiro para classificar Setor e Indústria de cada empresa. Entre elas:

- Yahoo Finance

- Google Finance

- Morningstar

- Reuters

Site institucional das empresas (especialmente para empresas privadas ou com classificação ambígua)

3. Classificação por Setor e Indústria
Useamos o modelo padrão de classificação GICS (Global Industry Classification Standard), que é utilizado por grandes instituições como a MSCI e a S&P. Ele divide as empresas em:
11 setores principais; 24 grupos industriais; 69 indústrias e 158 subindústrias

 Exemplos de aplicação:
- Zoom Video Communications (ZM)

Setor: Tecnologia

Indústria: Software

Justificativa: Atua oferecendo software de videoconferência em nuvem, sendo uma empresa 100% de base tecnológica.

- Tesla (TSLA)

Setor: Consumo Discricionário

Indústria: Fabricantes de Automóveis

Justificativa: Apesar de ter forte componente tecnológico, a maior parte da receita vem da venda de veículos elétricos.

- PayPal (PYPL)

Setor: Tecnologia

Indústria: Serviços de Tecnologia Financeira

Justificativa: Atuação primária é na intermediação de pagamentos digitais via tecnologia proprietária.

4. Atenção a Empresas com Múltiplas Atividades
Algumas empresas atuam em mais de um setor. Nesses casos, o critério foram:

- Qual atividade gera maior parte da receita

- Como a empresa é classificada oficialmente por serviços financeiros reconhecidos

